In [75]:
import re
from time import sleep
from selenium import webdriver # !pip install selenium
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd

In [6]:
adidas_df = pd.read_csv("data/adidas.csv") # first 4 pages

for i in range(5, 51):
    adidas_df = pd.concat([adidas_df, pd.read_csv(f"data/adidas_page{i}.csv")])

adidas_df.drop(columns = ["Unnamed: 0"], inplace = True)
print(f"Total number of shoes: {len(adidas_df)}")
display(adidas_df.head(10))

Total number of shoes: 2385


,title,subtitle,num_colors,url,price,reduced_price,description,details,colors,n_reviews,avg_stars
0,Start Your Run Shoes,Women's Running,4 colors,/us/start-your-run-shoes/GY9233.html,$65,$33,You'll want these adidas running shoes the nex...,NaN,Dash Grey / Matte Silver / Core Black,5.0,4.8
1,NMD_R1 Shoes,Youth Originals,NaN,/us/nmd_r1-shoes/H03994.html,$130,$91,"One shoe to rule them all. School, work or kic...",NaN,NaN,131.0,4.6
2,Edge Lux Shoes,Women's Training,5 colors,/us/edge-lux-shoes/GZ6741.html,$90,$45,"Comfort is key, whether you're racing to catch...",NaN,Core Black / Core Black / Iron Metallic,191.0,4.0
3,Adilette Comfort Slides,Sportswear,19 colors,/us/adilette-comfort-slides/GW9647.html,$40,$24,Classics for a reason. These adidas slides are...,NaN,Core Black / Core White / Grey Six,9735.0,4.7
4,Fluidflow 2.0 Shoes,Men's Sportswear,3 colors,https://www.adidas.com/us/fluidflow-2.0-shoes/...,$85,$51,It doesn't really matter whether or not a run ...,NaN,Legend Ink / Cloud White / Shadow Maroon,866.0,4.6
5,Adilette Comfort Adjustable Slides,Men's Essentials,2 colors,https://www.adidas.com/us/adilette-comfort-adj...,$40,$24,You know that feeling. When you slip off your ...,NaN,Core Black / Cloud White / Grey Six,1228.0,4.7
6,Lite Racer Adapt 5.0 Shoes,Men's Essentials,7 colors,https://www.adidas.com/us/lite-racer-adapt-5.0...,$70,$35,"Streamlined, sporty and super stylish, thanks ...",NaN,Cloud White / Core Black / Cloud White,96.0,4.7
7,Adilette Comfort Slides,Women's Sportswear,18 colors,https://www.adidas.com/us/adilette-comfort-sli...,$40,$20,From yoga on the beach directly to brunch? We'...,NaN,Signal Orange / Signal Orange / Vivid Berry,4941.0,4.8
8,Superstar Shoes,Women's Originals,15 colors,/us/superstar-shoes/FV3284.html,$95,$62,The adidas Superstar Shoes first stepped onto ...,NaN,Cloud White / Ice Mint / Cloud White,10524.0,4.7
9,NaN,NaN,NaN,/us/vulc-raid3r-shoes/GW9181.html,NaN,NaN,These adidas sneakers are tasteful and minimal...,NaN,Magic Mauve / Magic Mauve / Matt Purple Met.,168.0,4.5


In [15]:
# details column for the first 18 pages

details_df = pd.read_csv("data/adidas_page1_details.csv")

for i in range(2, 19):
    details_df = pd.concat([details_df, pd.read_csv(f"data/adidas_page{i}_details.csv")])

details_df.drop(columns = ["Unnamed: 0"], inplace = True)
display(details_df.head(5))

,title,subtitle,num_colors,url,price,description,details,colors,n_reviews,avg_stars,reduced_price
0,NMD_R1 Shoes,Youth Originals,40% off with code: cyber,/us/nmd_r1-shoes/H03994.html,$130,"One shoe to rule them all. School, work or kic...",Regular fit; Lace closure; Mesh upper; Boost m...,NaN,131.0,4.6,NaN
1,Lite Racer Adapt 5.0 Shoes,Men's Essentials,8 colors,/us/lite-racer-adapt-5.0-shoes/GX4675.html,$70,"Streamlined, sporty and super stylish, thanks ...",Regular fit; Slip-on with laces; Textile upper...,Cloud White / Core Black / Cloud White,99.0,4.7,$60
2,EQ21 Run 2.0 Bounce Sport Running Elastic Lace...,Children Sportswear,2 colors,/us/eq21-run-2.0-bounce-sport-running-elastic-...,$55,Those flowy lines aren't just for style. Desig...,Regular fit; Elastic laces with hook-and-loop ...,Royal Blue / Cloud White / Blue Rush,12.0,4.9,NaN
3,Adilette Comfort Slides,Women's Sportswear,18 colors,/us/adilette-comfort-slides/GZ6412.html,$40,From yoga on the beach directly to brunch? We'...,Regular fit; Slip-on construction; Synthetic u...,Signal Orange / Signal Orange / Vivid Berry,4948.0,4.8,$32
4,Adilette Comfort Adjustable Slides,Men's Essentials,2 colors,/us/adilette-comfort-adjustable-slides/EG1344....,$40,You know that feeling. When you slip off your ...,Regular fit; Hook-and-loop closure; Synthetic ...,Core Black / Cloud White / Grey Six,1232.0,4.7,$34


In [21]:
# replace the details column of the first 18 pages
adidas_fixed = adidas_df.merge(details_df[['url', 'details']], 
                               how = 'left',
                               suffixes = ("", "_right"),
                               on = 'url')
adidas_fixed.loc[adidas_fixed['details'].isna(), 'details'] = adidas_fixed['details_right']
adidas_fixed.drop(columns = ['details_right'], inplace = True)

display(adidas_fixed.head(10))

,title,subtitle,num_colors,url,price,reduced_price,description,details,colors,n_reviews,avg_stars
0,Start Your Run Shoes,Women's Running,4 colors,/us/start-your-run-shoes/GY9233.html,$65,$33,You'll want these adidas running shoes the nex...,Regular fit; Lace closure; Textile upper; LIGH...,Dash Grey / Matte Silver / Core Black,5.0,4.8
1,NMD_R1 Shoes,Youth Originals,NaN,/us/nmd_r1-shoes/H03994.html,$130,$91,"One shoe to rule them all. School, work or kic...",Regular fit; Lace closure; Mesh upper; Boost m...,NaN,131.0,4.6
2,Edge Lux Shoes,Women's Training,5 colors,/us/edge-lux-shoes/GZ6741.html,$90,$45,"Comfort is key, whether you're racing to catch...",Women's-specific fit; Lace closure; Textile up...,Core Black / Core Black / Iron Metallic,191.0,4.0
3,Adilette Comfort Slides,Sportswear,19 colors,/us/adilette-comfort-slides/GW9647.html,$40,$24,Classics for a reason. These adidas slides are...,Regular fit; Slip-on; Synthetic bandage upper;...,Core Black / Core White / Grey Six,9735.0,4.7
4,Fluidflow 2.0 Shoes,Men's Sportswear,3 colors,https://www.adidas.com/us/fluidflow-2.0-shoes/...,$85,$51,It doesn't really matter whether or not a run ...,Lace closure; Knit textile upper; Breathable f...,Legend Ink / Cloud White / Shadow Maroon,866.0,4.6
5,Adilette Comfort Adjustable Slides,Men's Essentials,2 colors,https://www.adidas.com/us/adilette-comfort-adj...,$40,$24,You know that feeling. When you slip off your ...,NaN,Core Black / Cloud White / Grey Six,1228.0,4.7
6,Lite Racer Adapt 5.0 Shoes,Men's Essentials,7 colors,https://www.adidas.com/us/lite-racer-adapt-5.0...,$70,$35,"Streamlined, sporty and super stylish, thanks ...",NaN,Cloud White / Core Black / Cloud White,96.0,4.7
7,Adilette Comfort Slides,Women's Sportswear,18 colors,https://www.adidas.com/us/adilette-comfort-sli...,$40,$20,From yoga on the beach directly to brunch? We'...,NaN,Signal Orange / Signal Orange / Vivid Berry,4941.0,4.8
8,Superstar Shoes,Women's Originals,15 colors,/us/superstar-shoes/FV3284.html,$95,$62,The adidas Superstar Shoes first stepped onto ...,Regular fit; Lace closure; Leather upper; Comf...,Cloud White / Ice Mint / Cloud White,10524.0,4.7
9,NaN,NaN,NaN,/us/vulc-raid3r-shoes/GW9181.html,NaN,NaN,These adidas sneakers are tasteful and minimal...,Lace closure; Canvas upper; Textile lining; Ru...,Magic Mauve / Magic Mauve / Matt Purple Met.,168.0,4.5


In [22]:
adidas_fixed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2390 entries, 0 to 2389
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          402 non-null    object 
 1   subtitle       402 non-null    object 
 2   num_colors     349 non-null    object 
 3   url            2390 non-null   object 
 4   price          399 non-null    object 
 5   reduced_price  259 non-null    object 
 6   description    2368 non-null   object 
 7   details        2248 non-null   object 
 8   colors         1945 non-null   object 
 9   n_reviews      2180 non-null   float64
 10  avg_stars      2180 non-null   float64
dtypes: float64(2), object(9)
memory usage: 288.6+ KB


In [80]:
# get title, num_colors, prices for the observations with missing titles

def get_title(page):
    title = page.find(attrs = {"testid": "product-title"})
    if title is None:
        return None
    return title.text

def get_subtitle(page):
    subtitle = page.find(attrs = {"data-auto-id": "product-category"})
    if subtitle is None:
        return None
    return subtitle.text
                               
def get_count(page):
    # Number of colors
    # e.g. "2 Colors"
    num_colors = page.select_one(".available-colors-label")
    if num_colors is None:
        return None
    num_colors = num_colors.text.replace(" available", "")
    return num_colors  
                               
def get_price(page):
    pricetag = page.select(".gl-price-item")
    return [p.text for p in pricetag] # usually starts with [original, sale]
    # may need to double check the order
                               
def get_page(url):
    driver = webdriver.Chrome()
    driver.get(url)
    sleep(3)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    driver.close()
    return soup

In [ ]:
for index, row in adidas_fixed.iterrows():
    if pd.isna(row['title']):
        abs_url = row['url']
        if "adidas.com" not in abs_url:
            abs_url = 'https://www.adidas.com' + row['url']
        soup = get_page(abs_url)
        row['title'] = get_title(soup)
        row['subtitle'] = get_subtitle(soup)
        row['num_colors'] = get_count(soup)
        price_listed = get_price(soup)
        if len(price_listed) == 1: 
            row['price'] = price_listed[0]
        elif len(price_listed) == 2:
            row['price'] = price_listed[0]
            row['reduced_price'] = price_listed[1]
        elif len(price_listed) == 0:
            row['price'] = None # missing price

In [ ]:
# investigate number of missing values again after more scraping
adidas_fixed.info()

In [ ]:
## if num_colors is missing, likely there's only one color
## So, get it from the details column
adidas_fixed[adidas_fixed['num_colors'].isna(), 'num_colors'] = 1

def get_one_color(text):
    index_start = text.index("Product color: ")
    index_end = text.index("Product color: ", start = index_start)
    color = text[index_start + 15: index_end]
    return color

adidas_fixed[adidas_fixed['num_colors'].isna(), 'colors'] = \
    adidas_fixed['details'].apply(get_one_color)

In [ ]:
## extract product code
def get_product_code(url):
    index_start = url.rindex("/")
    index_end = url.rindex(".html")
    return text[index_start + 1: index_end]
    
adidas_fixed['product_code'] = adidas_fixed['url'].apply(get_product_code)

In [ ]:
adidas_fixed.info()

In [ ]:
adidas_fixed.to_csv("adidas_merged.csv")